In [26]:
class CNF:
    def __init__(self, n_var, n_clauses, clauses):
        self.var_map = {}
        self.n_var = n_var
        self.n_clauses = n_clauses
        self.clauses = clauses
        self.satisfiable = 0 # unknown -> 0, sat -> 1, unsat -> 2

In [27]:
def make_formula(filename):
    f = open(filename)
    lines = f.readlines()
    flag = 0
    clauses = []
    n_var = 0
    n_clauses = 0
    for line in lines:
        if (flag == 0):
            if (line[0] == 'p'):
                flag = 1
                info = line.split()
                n_var = info[2]
                n_clauses = info[3]
        else:
            clause = list(map(int, line.split()[:-1]))
            clauses.append(clause)
    return CNF(n_var, n_clauses, clauses)


In [61]:
cnf = make_formula('test1.cnf')
cnf2 = do_pure_literal(cnf)
print(cnf2.clauses)

[-1, 5, 3, -4]
[[1, -5]]


In [39]:
def assign_value(n):
    if (n < 0):
        return 0
    else:
        return 1

In [40]:
def find_var_in_clause(clause, var):
    for v in clause:
        if abs(v) == var:
            if (v < 0):
                return 0
            else:
                return 1

In [63]:
def resolution_for_var(c1, c2, var):
    new_c1 = c1[:]
    new_c2 = c2[:]
    if var in c1:
        new_c1.remove(var)
    else:
        new_c1.remove(-var)
    if var in c2:
        new_c2.remove(var)
    else:
        new_c2.remove(-var)
    return list(set(new_c1 + new_c2))

In [ ]:
def assign_val_to_var(var_map, clauses)
for var in var_map.keys():
    for clause in clauses:
        clause_search = list(map(abs, clause[:]))
        if var in clause_search:
            assign = cnf.var_map[var]
            if (assign == find_var_in_clause(clause, var)):
                new_clauses_edit.remove(clause)
            else:
                index = new_clauses_edit.index(clause)
                if assign == 1:
                    v = -var
                else:
                    v = var
                new_clauses_edit[index].remove(v)

In [60]:
def do_pure_literal(cnf):
    clauses = cnf.clauses
    new_clauses = []
    new_clauses_edit = []
    unit_var = 0
    unit_index = []
    for i, clause in enumerate(clauses):
        if (len(clause) == 1):
            unit_var = clause[0]
            if (unit_var < 0):
                if not (check_var_map(cnf.var_map, -unit_var, 0)):
                    cnf.satisfiable = 2
                cnf.var_map[-unit_var] = 0
                unit_index.append(i)
            else:
                if not (check_var_map(cnf.var_map, unit_var, 1)):
                    cnf.satisfiable = 2
                cnf.var_map[unit_var] = 1
                unit_index.append(i)
    for i in range(len(clauses)):
        if not (i in unit_index):
            new_clauses.append(clauses[i])
    new_clauses_edit = new_clauses[:]
    for var in cnf.var_map.keys():
        for clause in new_clauses:
            clause_search = list(map(abs, clause[:]))
            if var in clause_search:
                assign = cnf.var_map[var]
                if (assign == find_var_in_clause(clause, var)):
                    new_clauses_edit.remove(clause)
                else:
                    index = new_clauses_edit.index(clause)
                    if assign == 1:
                        v = -var
                    else:
                        v = var
                    new_clauses_edit[index].remove(v)
    cnf.clauses = new_clauses_edit
    cnf.n_clauses = len(new_clauses)
    if (cnf.n_clauses == 0):
        cnf.satisfiable = 1
    return cnf

In [46]:
def check_var_map(a, var, val):
    if (var in a):
        if (a[var] != val):
            return False
        else:
            return True
    else:
        return True

In [31]:
f = open('test1.cnf')
lines = f.readlines()
flag = 0
clauses = []
for i, line in enumerate(lines):
    if (flag == 0):
        print(line[0])
        if (line[0] == 'p'):
            flag = 1
            k = line.split()
            print(k)
    else:
        clause = list(map(int, line.split()))
        clause = clause[:-1]
        clauses.append(clause)
print(clauses)
        

c
c
p
['p', 'cnf', '5', '3']
[[1, -5, 4], [-1, 5, 3, 4], [-4]]
